In [1]:
# Downloading tiny shakespeare dataset...

!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-06-13 13:01:44--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8003::154, 2606:50c0:8000::154, 2606:50c0:8001::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8003::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.1’

input.txt.1         100%[===================>]   1.06M  6.74MB/s    in 0.2s    

2024-06-13 13:01:47 (6.74 MB/s) - ‘input.txt.1’ saved [1115394/1115394]



In [2]:
# Reading and Inspecting the Data
with open('input.txt', 'r') as file:
    data = file.read()

In [3]:
print("length of dataset in characters: ", len(data))

length of dataset in characters:  1115394


In [4]:
# Looking at the first 1000 characters
print(data[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [5]:
# All unique characters in the dataset
chars = sorted(list(set(data)))
vocab_size = len(chars)
print(''.join(chars))
print('Number of unique characters: ', vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
Number of unique characters:  65


In [6]:
# create a mapping of unique characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }

encode = lambda s: [stoi[ch] for ch in s] # encoder: take a string and convert it into a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers and convert it into a string

# Encoding the dataset
print(encode('hello'))
print(decode(encode('hello')))

[46, 43, 50, 50, 53]
hello


In [7]:
# lets encode the entire dataset and store it into a torch.Tensor
import torch

data_ = torch.tensor(encode(data), dtype=torch.long)
print(data_.shape, data_.dtype)
print(data_[:1000]) # first 100 characters

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [8]:
# Lets now split the dataset into training and validation sets
n = int(0.9*len(data_)) # 90% of the data for training and 10% for validation
train_data, valid_data = data_[:n], data_[n:]

In [9]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [10]:
x = train_data[:block_size]
y = train_data[1:block_size+1] #next block size characters offset by 1
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} then target is {target}")

when input is tensor([18]) then target is 47
when input is tensor([18, 47]) then target is 56
when input is tensor([18, 47, 56]) then target is 57
when input is tensor([18, 47, 56, 57]) then target is 58
when input is tensor([18, 47, 56, 57, 58]) then target is 1
when input is tensor([18, 47, 56, 57, 58,  1]) then target is 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) then target is 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) then target is 58


In [11]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of input x and targets y
    data = train_data if split == 'train' else valid_data
    ix = torch.randint(len(data) - block_size, (batch_size,)) # starting index for each sequence
    x = torch.stack([data[i:i+block_size] for i in ix]) # input data
    y = torch.stack([data[i+1:i+block_size+1] for i in ix]) # target data
    return x, y

xb, yb = get_batch('train')
print("Inputs: ")
print(xb.shape)
print(xb)
print('targets: ')
print(yb.shape)
print(yb)

print('---'*20)

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f"when is {context.tolist()} then target is {target}")

Inputs: 
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets: 
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
------------------------------------------------------------
when is [24] then target is 43
when is [24, 43] then target is 58
when is [24, 43, 58] then target is 5
when is [24, 43, 58, 5] then target is 57
when is [24, 43, 58, 5, 57] then target is 1
when is [24, 43, 58, 5, 57, 1] then target is 46
when is [24, 43, 58, 5, 57, 1, 46] then target is 43
when is [24, 43, 58, 5, 57, 1, 46, 43] then target is 39
when is [44] then target is 53
when is [44, 53] then target is 56
when is [44, 53, 56] then target is 1
when is [44, 53, 56, 1] then target is 58
when is [44, 53, 56, 1, 58] then target is 46
wh

In [12]:
print(xb) # our input to the model

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])


In [13]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    
    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, idx, targets=None):
        
        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (Batch, Time, Channels)
        # (B, T, C) -> (B, C, T)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # becomes (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # becomes (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # becomes (B, T+1)
        return idx
    
model = BigramLanguageModel(vocab_size)
logits, loss = model(xb, yb)
print(logits.shape)
print(loss) # - ln(1/65) = 4.8758

idx = torch.zeros((1, 1), dtype=torch.long)
print(decode(model.generate(idx=idx, max_new_tokens=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)

SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


In [14]:
# create a Pytorch optimizer    
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

/Users/aakashvardhan/.pyenv/versions/3.11.0/envs/myproject/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
batch_size = 32

for steps in range(10000):
    # sample a batch of data
    xb, yb = get_batch('train')
    
    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.382369041442871


In [16]:
print(decode(model.generate(idx=idx, max_new_tokens=500)[0].tolist()))


lso br. ave aviasurf my, yxMPZI ivee iuedrd whar ksth y h bora s be hese, woweee; the! KI 'de, ulseecherd d o blllando;LUCEO, oraingofof win!
RIfans picspeserer hee tha,
TOFonk? me ain ckntoty ded. bo'llll st ta d:
ELIS me hurf lal y, ma dus pe athouo
BEY:! Indy; by s afreanoo adicererupa anse tecorro llaus a!
OLeneerithesinthengove fal amas trr
TI ar I t, mes, n IUSt my w, fredeeyove
THek' merer, dd
We ntem lud engitheso; cer ize helorowaginte the?
Thak orblyoruldvicee chot, p,
Bealivolde Th li


In [17]:
print(decode(model.generate(idx=idx, max_new_tokens=1000)[0].tolist()))


Fo beamen, tofr,
n s Byo tred ceathe, il ivilde w
O ff y
Fivede? ig aiMy, I ivis muofounce herevern outh f athawendesees yof th withS:

FiFLINR:

Wheader y blitow,
Ye m o ditoshyd me, ch rte u hart ararwsa
Wou fe,
INurathoune
IESSARin,
MIOLened sus;
Wh.
S:
NMy BAnind g.
iudshank
An chin is a arokisupxaseru t w ity merwo al LOLo bebte loolld worinero ya l aknge ond thal ttry b's mo ge ck.

gh, inketilllin trk$nutud t ar,
WAnt cithap's Zimponcrdistherdrtes saure ' erpoperrposthel?
Handis of hef thep: ct
Ywit harfoul'st, ar izlor t ct.
Fo, sther:
I d tre th,-she.
Wowltothedl:
NNONANRI, aft,
STo way!
TIV:
WDUKI HANENEThe d ndean-bros g qpl mout fok yolaime do myoulato,
Mok h$ay t nch sle bionhoured whaneables mye.
For f beng tho; ar!TCald? min, wherur thaing tyoucora we d s?

Tord, g I:
Whireat pr f Yhzze?
Ther hurer cr il f aloulatspribr,
AG o otr thall oull
NEN theat.
We

GAseruisfeveg, t wild he borong s bl?
DW:
Anto' h bes whoommeulye Werngssamyou
She
BESCONG RDoleshet heked
GUE t at,

### The mathematical trick in self-attention

In [18]:
# Toy Example:

torch.manual_seed(1337)
B,T,C = 4,8,2 # batch size, time steps, channels
x = torch.randn(B,T,C) # input data
x.shape

torch.Size([4, 8, 2])

In [19]:
# We want x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev, 0)

In [20]:
x[0]

tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])

In [21]:
xbow[0] # while first element is same as x, the rest are different as they are the mean of the previous elements in x.

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

#### Efficient approach

In [27]:
# version 2: weighted sum
wei = torch.tril(torch.ones(T,T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B,T,T) @ (B,T,C) -> (B,T,C)
torch.allclose(xbow, xbow2)

True

In [23]:
torch.tril(torch.ones(3,3))

tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])

In [37]:
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei

tensor([[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0., 0., 0., 0.]])

In [38]:
# version 3: use Softmax
tril = torch.tril(torch.ones(T,T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)

True

In [39]:
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [47]:
k[0:2][0]

tensor([[ 0.1196, -0.3013,  0.3629,  1.1771,  1.1385, -0.2554,  0.1454, -0.2944,
         -0.7020, -1.0308,  0.7436, -0.8098, -0.6669,  0.0912, -0.0061,  0.1983],
        [-0.5423, -0.5558, -0.0761,  1.2929,  0.8653, -1.1998,  0.3878,  0.1939,
          0.7024, -0.8225,  0.2348, -0.8499, -0.3813, -0.2991,  0.0102, -0.5545],
        [-0.3736, -0.4678, -0.2156, -0.8034, -0.3715, -0.5443, -0.9146, -0.0559,
         -0.3290, -0.2102,  0.1166, -0.1798, -0.2820, -0.3320, -0.4596, -0.1325],
        [-0.3146,  0.0845, -0.1235, -0.7058, -0.1802,  0.5492, -0.8980, -0.4938,
          0.6791,  0.8827,  0.4911,  0.5190,  0.9011,  0.0913, -0.1933, -0.6770],
        [ 0.0239,  0.0998, -0.1871, -0.0860, -0.4881, -1.6765,  0.2413,  0.7361,
          0.4608, -0.8722, -0.4259, -1.1347, -1.0571, -0.9401,  0.1343, -0.0157],
        [-0.2362, -0.7873, -0.3802,  0.5815, -0.3722,  1.2405, -0.7004, -1.4917,
          0.7678,  0.3584,  0.6120, -0.0794,  0.5983,  0.2635,  0.6490,  0.0709],
        [-0.7941, -0.1

In [48]:
k[0:2][0].transpose(-2, -1)

tensor([[ 0.1196, -0.5423, -0.3736, -0.3146,  0.0239, -0.2362, -0.7941,  0.1651],
        [-0.3013, -0.5558, -0.4678,  0.0845,  0.0998, -0.7873, -0.1660, -0.1599],
        [ 0.3629, -0.0761, -0.2156, -0.1235, -0.1871, -0.3802, -0.2810, -0.5717],
        [ 1.1771,  1.2929, -0.8034, -0.7058, -0.0860,  0.5815, -0.1021, -0.3957],
        [ 1.1385,  0.8653, -0.3715, -0.1802, -0.4881, -0.3722, -0.7352,  0.3930],
        [-0.2554, -1.1998, -0.5443,  0.5492, -1.6765,  1.2405, -0.7518, -0.8567],
        [ 0.1454,  0.3878, -0.9146, -0.8980,  0.2413, -0.7004, -0.1276,  0.3390],
        [-0.2944,  0.1939, -0.0559, -0.4938,  0.7361, -1.4917, -0.0051, -0.7977],
        [-0.7020,  0.7024, -0.3290,  0.6791,  0.4608,  0.7678,  0.3325,  0.2213],
        [-1.0308, -0.8225, -0.2102,  0.8827, -0.8722,  0.3584, -0.3374, -0.5161],
        [ 0.7436,  0.2348,  0.1166,  0.4911, -0.4259,  0.6120,  0.1678,  0.1850],
        [-0.8098, -0.8499, -0.1798,  0.5190, -1.1347, -0.0794,  0.3105, -0.2105],
        [-0.6669

In [54]:
# version 4: self-attention!
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch size, time steps, channels
x = torch.randn(B,T,C) # input data

# lets see a single head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)

k = key(x) # (B,T,C) -> (B,T,H) H= head_size
q = query(x) # (B,T,C) -> (B,T,H)
wei = q @ k.transpose(-2,-1) # (B,T,H) @ (B,H,T) -> (B,T,T)

tril = torch.tril(torch.ones(T,T))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)


v = value(x) # (B,T,C) -> (B,T,H)
# out = wei @ x
out = wei @ v # (B,T,T) @ (B,T,H) -> (B,T,H)

out.shape

torch.Size([4, 8, 16])

- Attention is a communication mechanism. Can be seen as nodes in a directed graph looking at each other and aggregating information with a weighted sum from all nodes that point to them, with data-dependent weights.
- There is no notion of space. Attention simply acts over a set of vector. This is why need to positionally encode tokens.
- Each example across batch dimenssion is of course processed independently and never interact with each other.
- In an encoder attention block just delete the single line that does masking with tril, allowing all tokens to communicate. This block here is called a "decoder" attention block because it has triangular masking, and is usually used in autoregressive settings, like language modelling.
- "self-attention" just means that the keys and values are produced in the same source as queries. In "cross-attention", the queries still get produced from x, but the keys and values come from some other, external source (e.g. an encoder module)
- "Scaled" attention additionally divides `wei` by 1/sqrt(head_size). This makes it so when input Q,K are unit variance, wei will be unit variance too and softmax will stay diffuse and not saturate too much.

In [55]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)

In [56]:
k = torch.randn(B,T, head_size)
q = torch.randn(B,T, head_size)
wei = q @ k.transpose(-2,-1) * (head_size ** -0.5)

In [57]:
k.var()

tensor(1.0449)

In [58]:
q.var()

tensor(1.0700)

In [60]:
wei.var()

tensor(1.0918)

In [61]:
torch.softmax(torch.tensor([0.1,-0.2,0.3,-0.2,0.5]), dim=-1)

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])

In [62]:
torch.softmax(torch.tensor([0.1,-0.2,0.3,-0.2,0.5])*8, dim=-1) # scaling/sharpen the values by 8

tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])

In [25]:
torch.manual_seed(42)
a = torch.tril(torch.ones(3,3))
a = a / torch.sum(a, dim=1, keepdim=True)
b = torch.randint(0, 10, (3,2)).float()
c = a @ b
print('a=')
print(a)
print('-'*8)
print('b=')
print(b)
print('-'*8)
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--------
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--------
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [63]:
class BatchNorm1d:
    
    def __init__(self, dim, eps=1e-5, momentum=0.1):
        self.eps = eps
        self.gamma = torch.ones(dim)
        self.beta = torch.zeros(dim)
        
    def __call__(self, x):
        # calculate the forward pass
        xmean = x.mean(1, keepdim=True) # batch mean
        xvar = x.var(1, keepdim=True) # batch variance
        xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
        self.out = self.gamma * xhat + self.beta
        return self.out
    
    def parameters(self):
        return [self.gamma, self.beta]
    
torch.manual_seed(1337)
module = BatchNorm1d(100)
x = torch.randn(32, 100)
x = module(x)
x.shape

torch.Size([32, 100])

In [64]:
x[:,0].mean(), x[:,0].std()

(tensor(0.1469), tensor(0.8803))

In [65]:
x[0,:].mean(), x[0,:].std()

(tensor(-3.5763e-09), tensor(1.0000))